## Preprocessing

In [ ]:
#IMPORTS
import pandas as pd
import numpy as np

In [1]:
with open('transcript.txt', 'r') as file:
    transcript = [line.strip() for line in file if line.strip() != '']

In [4]:
import pandas as pd

# Split each line at the first colon
data = []
for line in transcript:
    if ':' in line:
        speaker, text = line.split(':', 1)
        text = text.lstrip()  # Remove leading space
        data.append([speaker, text])
    else:
        data.append(['', line])

transcript_df = pd.DataFrame(data, columns=['Speaker', 'Text'])
transcript_df.head()

,Speaker,Text
0,DAVID MUIR,"Tonight, the high-stakes showdown here in Phil..."
1,LINSEY DAVIS,A historic race for president upended just wee...
2,DAVID MUIR,The candidates separated by the smallest of ma...
3,DAVID MUIR,"Good evening, I'm David Muir. And thank you fo..."
4,LINSEY DAVIS,And I'm Linsey Davis. Tonight's meeting could ...


## First Test - BERT

In [6]:
#!pip install pandas nltk transformers torch

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np